# Import Statements and Magic Commands

In [1]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree
#from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
#from xgboost import XGBRegressor
#from sklearn import linear_model
#from sklearn.metrics import mean_squared_log_error
# WHEN CHECKING THE MODEL: use as np.sqrt(mean_squared_log_error( y_test, predictions ))
from pandas.tseries.holiday import USFederalHolidayCalendar
%matplotlib inline

---
# Load the Data

### Building and Weather Data

In [6]:
# Note: These assume that the data has been saved to a subdirectory named "energy"
bldgData = pd.read_csv("energy/building_metadata.csv")
weatherTrain = pd.read_csv("energy/weather_train.csv")
weatherTest = pd.read_csv("energy/weather_test.csv")
allWeather = weatherTrain.append(weatherTest)

### Training Data

In [7]:
energyTrain = pd.read_csv("energy/train.csv")

### Test Data
Warning: This will use a lot of memory! My virtual machine is sitting at 5.5 GB of RAM utilization right now.

In [8]:
energyTest = pd.read_csv("energy/test.csv")

### Explore the Data

In [4]:
#print(bldgData.head(5))
#print(allWeather.head(5))
#print(allWeather.tail(5))
print(energyTrain.head(5))

   building_id  meter            timestamp  meter_reading
0            0      0  2016-01-01 00:00:00            0.0
1            1      0  2016-01-01 00:00:00            0.0
2            2      0  2016-01-01 00:00:00            0.0
3            3      0  2016-01-01 00:00:00            0.0
4            4      0  2016-01-01 00:00:00            0.0


---
# 5. Averaging

In [2]:
dtr1results = pd.read_csv("results/decisionTreeRegressor02_merged.csv")
dtr2results = pd.read_csv("results/decisionTree01.csv")

dtr2results.rename(columns={"meter_reading":"meter_reading2"}, inplace=True)
dtr2results.head(5)


,row_id,meter_reading2
0,0,54.832333
1,1,19.384700
2,2,2.047667
3,3,113.418667
4,4,472.330000


In [4]:
mergedResults = pd.merge(dtr1results, dtr2results, on='row_id', how='left')
mergedResults.tail(5)

,row_id,meter_reading,meter_reading2
41697595,41697595,4.4750,6.290476
41697596,41697596,5.3375,5.140476
41697597,41697597,10.7750,6.105952
41697598,41697598,170.0938,164.836905
41697599,41697599,4.4625,4.365476


In [5]:
mergedResults.head(10)

,row_id,meter_reading,meter_reading2
0,0,37.5408,54.832333
1,1,14.1973,19.384700
2,2,1.6381,2.047667
3,3,336.5020,113.418667
4,4,86.0026,472.330000
5,5,4.4366,5.801767
6,6,99.1077,45.322000
7,7,526.9370,163.928333
8,8,886.4180,1055.260000
9,9,478.1330,146.750333


In [6]:
mergedResults['average'] = mergedResults[['meter_reading','meter_reading2']].mean(axis=1)
mergedResults.head(10)

,row_id,meter_reading,meter_reading2,average
0,0,37.5408,54.832333,46.186567
1,1,14.1973,19.384700,16.791000
2,2,1.6381,2.047667,1.842883
3,3,336.5020,113.418667,224.960333
4,4,86.0026,472.330000,279.166300
5,5,4.4366,5.801767,5.119183
6,6,99.1077,45.322000,72.214850
7,7,526.9370,163.928333,345.432667
8,8,886.4180,1055.260000,970.839000
9,9,478.1330,146.750333,312.441667


In [7]:
mergedResults.drop("meter_reading", axis=1, inplace=True)
mergedResults.drop("meter_reading2", axis=1, inplace=True)
mergedResults.rename(columns={"average":"meter_reading"}, inplace=True)
mergedResults.head(10)

,row_id,meter_reading
0,0,46.186567
1,1,16.791000
2,2,1.842883
3,3,224.960333
4,4,279.166300
5,5,5.119183
6,6,72.214850
7,7,345.432667
8,8,970.839000
9,9,312.441667


In [8]:
mergedResults.tail(5)

,row_id,meter_reading
41697595,41697595,5.382738
41697596,41697596,5.238988
41697597,41697597,8.440476
41697598,41697598,167.465352
41697599,41697599,4.413988


In [9]:
mergedResults.to_csv("results/average_of_universal_and_site_specific.csv", index=False)

### Kaggle RMSLE score for the Averaged results = 1.29